In [102]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [111]:
#import "Annual_Parking_Study_Data" to creat 'pkdt' dataframe
#read data
def read_data(filename, col_names):
    df = pd.read_csv(filename)
    df = df[col_names]
    return df

col_names = ['Date Time','Unitdesc','Side', 'Parking_Spaces', 'Total_Vehicle_Count', 'Construction', 'Event Closure']
pkdt = read_data('Annual_Parking_Study_Data.csv', col_names)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [104]:
#specific processing
pkdt.rename(columns={'Event Closure' : 'Event', 'Date Time' : 'Time'}, inplace = True)
pkdt.reset_index(drop=True, inplace=True)
pkdt = pkdt.dropna()

In [105]:
#filter the data with 'Construction' and 'Event' column are "No"
pkdt = pkdt[pkdt.Construction != 'Yes']
pkdt = pkdt[pkdt.Construction != 'yes']
pkdt = pkdt[pkdt.Construction != 'ERROR: #N/A']
pkdt = pkdt[pkdt.Event != 'Yes']
pkdt = pkdt.drop('Construction', axis = 1)
pkdt = pkdt.drop('Event', axis = 1)

In [106]:
#group rows with same 'Time' and 'Unitdesc', for other columns, find the sum
pkdt = pkdt.groupby(['Time', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Total_Vehicle_Count'].sum()

In [107]:
#find the occupancy and append it to the pkdt
Occupancy = np.minimum(pkdt['Total_Vehicle_Count'] / pkdt['Parking_Spaces'], 1)
pkdt['Occupancy'] = Occupancy
pkdt = pkdt.dropna()
pkdt = pkdt.drop('Total_Vehicle_Count', axis = 1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in minimum
  


In [108]:
#retrivev 'hour' column from general datetime
pkdt['Time'] = pd.to_datetime(pkdt['Time'], format = '%m-%d-%y %H:%M:%S')
times = pd.DatetimeIndex(pkdt['Time'])
pkdt['Hour'] = times.hour


In [109]:
#group data by 'hour' and 'Unidesc' and average other columns
pkdt = pkdt.groupby(['Hour', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Occupancy'].mean()

In [110]:
#select specific street name
def select_street(df, street_name):
    df = df[df['Unitdesc'] == street_name]
    df = df.groupby(['Hour', 'Unitdesc'] ,as_index = False)['Parking_Spaces', 'Occupancy'].mean()
    return df

select_street(pkdt, '10TH AVE BETWEEN E PIKE ST AND E PINE ST')

,Hour,Unitdesc,Parking_Spaces,Occupancy
0,0,10TH AVE BETWEEN E PIKE ST AND E PINE ST,53.333333,0.983333
1,1,10TH AVE BETWEEN E PIKE ST AND E PINE ST,40.000000,0.350000
2,2,10TH AVE BETWEEN E PIKE ST AND E PINE ST,40.000000,0.200000
3,8,10TH AVE BETWEEN E PIKE ST AND E PINE ST,46.800000,0.493407
4,9,10TH AVE BETWEEN E PIKE ST AND E PINE ST,46.800000,0.626703
5,10,10TH AVE BETWEEN E PIKE ST AND E PINE ST,46.800000,0.829231
6,11,10TH AVE BETWEEN E PIKE ST AND E PINE ST,46.800000,0.795073
7,12,10TH AVE BETWEEN E PIKE ST AND E PINE ST,46.800000,0.929872
8,13,10TH AVE BETWEEN E PIKE ST AND E PINE ST,46.800000,0.906630
9,14,10TH AVE BETWEEN E PIKE ST AND E PINE ST,46.800000,0.881502
